In [67]:
#importing dependecies
import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig, Trainer, TrainingArguments

In [68]:
#creatin dataset
dataset1=pd.read_parquet(r"C:\Users\noman\internship projects\text summarizer\dataset_cnn\cnn_dailymail-train-00000-of-00003.parquet", engine='fastparquet')
dataset2=pd.read_parquet(r"C:\Users\noman\internship projects\text summarizer\dataset_cnn\cnn_dailymail-train-00001-of-00003.parquet", engine='fastparquet')
dataset3=pd.read_parquet(r"C:\Users\noman\internship projects\text summarizer\dataset_cnn\cnn_dailymail-train-00002-of-00003.parquet", engine='fastparquet')

In [69]:
#merging dataset
dataset=dataset1+dataset2+dataset3
dataset

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4a03a18...
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9906a80...
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da3712b801...
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88904093...
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a94fd1a...
...,...,...,...
115995,NaN,NaN,NaN
115996,NaN,NaN,NaN
115997,NaN,NaN,NaN
115998,NaN,NaN,NaN


In [70]:
#extracting a particular data point
print(dataset.iloc[10000]['article'])

(CNN) -- Costa Rica elected its first female president, as the ruling National Liberation Party claimed a historic victory. "I want to thank the pioneering women who years ago opened the doors of politics in Costa Rica," Laura Chinchilla said Sunday to flag-waving supporters in the capital, San Jose. "My government will be open to all Costa Ricans of good faith." Second-place candidate Otton Solis of the Citizen's Action Party had conceded defeat earlier in the evening, as the PLN -- the Spanish acronym for the National Liberation Party party -- forged its way to a nearly 2-to-1 lead. Polls showed Chinchilla garnering 46.8 percent of the vote, with 84 percent of election sites reporting. Solis had 25.1 percent, while Otto Guevara of the Libertarian Movement had 20.9 percent. Election day was festive throughout the nation, with video footage showing supporters of the main presidential candidates dressed in their respective campaign colors. International observers interviewed on CNN affi

In [71]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 116000 entries, 0 to 115999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article     55113 non-null  object
 1   highlights  55113 non-null  object
 2   id          55113 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


In [72]:
#Dropping duplicate and null,na,nan values
dataset.drop_duplicates(inplace=True)
dataset.dropna(inplace=True)

dataset.isna().sum()

article       0
highlights    0
id            0
dtype: int64

In [73]:
#dataset for validation
validation_data=pd.read_parquet(r"C:\Users\noman\internship projects\text summarizer\dataset_cnn\cnn_dailymail-validation.parquet", engine='fastparquet')
val1=list(validation_data["article"])
val2=list(validation_data["highlights"])

In [74]:
data1=list(dataset["article"][0:1000])
data2=list(dataset["highlights"][0:1000])

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [78]:
#tokenizing our training data
tokenizer=BartTokenizer.from_pretrained("facebook/bart-base", num_threads=8)
tokens_train=tokenizer.encode_plus(data1, data2, truncation=True, padding=True, max_length=26739)

In [79]:
#tokenizing our validation data
tokens_val=tokenizer.encode_plus(val1, val2, truncation=True, padding=True, max_length=26739, return_tensors='pt')

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [80]:
#initializing the model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model=model.to(device)

In [81]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir=r"C:\Users\noman\internship projects\text summarizer\output",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_steps=100,
    save_strategy="epoch",
    optim="adamw_torch"
)

In [82]:
# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokens_train,
    eval_dataset=tokens_val,
)

In [83]:
# Train the model
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'